# Experimentos: Baseline com QSVM

Este notebook estabelece baselines de performance com QSVM em datasets

# Dataset = Breast Cancer Wisconsin (Diagnostic)

In [40]:
from sklearn.datasets import load_breast_cancer 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
import numpy as np
import pandas as pd

#Ferramentas quânticas 
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import StatevectorSampler    
from qiskit_machine_learning.kernels import FidelityQuantumKernel  
from qiskit_machine_learning.algorithms import QSVC
from sklearn.metrics import classification_report
from sklearn.datasets import load_breast_cancer
import time

## Loading data and initial analysis

In [41]:
X, y = load_breast_cancer(return_X_y=True)  # Carregando o dataset

# Verificando as dimensões dos nossos dados
print(f"Dimensões dos dados: {X.shape}")


# Vendo a distribuição das classes (0 e 1)
print("\nDistribuição das classes:")
# Usando numpy para contar as ocorrências de cada classe
unique, counts = np.unique(y, return_counts=True)
for class_label, count in zip(unique, counts):
    print(f"Classe {class_label}: {count} ocorrências")


Dimensões dos dados: (569, 30)

Distribuição das classes:
Classe 0: 212 ocorrências
Classe 1: 357 ocorrências


In [ ]:
X_reduced = X[:, :10] 

idx_class_0 = np.where(y == 0)[0][:100]
idx_class_1 = np.where(y == 1)[0][:100]

selected_idx = np.concatenate([idx_class_0, idx_class_1])
np.random.shuffle(selected_idx)

X_small = X_reduced[selected_idx]
y_small = y[selected_idx]



## Split data into training and testing sets

In [43]:
X_train, X_test, y_train, y_test = train_test_split(
    X_small, y_small, test_size=0.25, random_state=42, stratify=y_small
)
print(f"Dimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")


Dimensões do conjunto de treino: (75, 10)
Dimensões do conjunto de teste: (25, 10)


## Normalize data

In [44]:
# normalizing the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)  # Ajusta o scaler e transforma os dados de treino
X_test = scaler.transform(X_test)  # Transforma os dados de teste com o mesmo scaler

print("\nDados normalizados")






Dados normalizados


# Training QSVM model

In [45]:
# training qsvm model

num_features = X_train.shape[1]

# Definindo o mapeamento de características
feature_map_bc = ZZFeatureMap(feature_dimension=num_features, reps=1)

# Criando o kernel quântico
fidelity_kernel_bc = FidelityQuantumKernel(feature_map=feature_map_bc)

# Instanciando o modelo QSVC
qsvc_bc = QSVC(quantum_kernel=fidelity_kernel_bc, random_state=42)

# Treinando o modelo
start_time = time.time()
qsvc_bc.fit(X_train, y_train)
end_time = time.time()
print(f"Modelo treinado em {end_time - start_time:.2f} segundos.")

# Fazendo previsões
qsvc_predictions_bc = qsvc_bc.predict(X_test)

# Avaliando o modelo
print("\n--- Relatório de Classificação para o QSVM (Breast Cancer) ---")
print(classification_report(y_test, qsvc_predictions_bc, target_names=load_breast_cancer().target_names))

Modelo treinado em 135.73 segundos.

--- Relatório de Classificação para o QSVM (Breast Cancer) ---
              precision    recall  f1-score   support

   malignant       0.78      0.54      0.64        13
      benign       0.62      0.83      0.71        12

    accuracy                           0.68        25
   macro avg       0.70      0.69      0.68        25
weighted avg       0.70      0.68      0.67        25



# Dataset = Student Performance

In [46]:
"""# Importando o dataset
student_performance = 'student-por.csv'

df_student_performance = pd.read_csv(student_performance, sep=',')  # Lendo o dataset com separador ','

# 3. Replicamos a separação que a biblioteca fazia para nós:
# y são as colunas de notas
y = df_student_performance[['G1', 'G2', 'G3']]
# X são TODAS as outras colunas
X = df_student_performance.drop(columns=['G1', 'G2', 'G3'])

# --- Verificações para confirmar que deu tudo certo ---
print("--- Dataset carregado localmente com sucesso! ---")
print("Formato das features (X):", X.shape)
print("Formato dos alvos (y):", y.shape)

print("\n5 primeiras linhas das features (X):")
"""

'# Importando o dataset\nstudent_performance = \'student-por.csv\'\n\ndf_student_performance = pd.read_csv(student_performance, sep=\',\')  # Lendo o dataset com separador \',\'\n\n# 3. Replicamos a separação que a biblioteca fazia para nós:\n# y são as colunas de notas\ny = df_student_performance[[\'G1\', \'G2\', \'G3\']]\n# X são TODAS as outras colunas\nX = df_student_performance.drop(columns=[\'G1\', \'G2\', \'G3\'])\n\n# --- Verificações para confirmar que deu tudo certo ---\nprint("--- Dataset carregado localmente com sucesso! ---")\nprint("Formato das features (X):", X.shape)\nprint("Formato dos alvos (y):", y.shape)\n\nprint("\n5 primeiras linhas das features (X):")\n'

In [47]:
#X.head(5) 

In [48]:
"""# Dimensões dos dados do Student Performance Dataset
print("\nDimensões do Student Performance Dataset:")
print(f"X: {X.shape}")
print(f"y: {y.shape}")"""


'# Dimensões dos dados do Student Performance Dataset\nprint("\nDimensões do Student Performance Dataset:")\nprint(f"X: {X.shape}")\nprint(f"y: {y.shape}")'

In [49]:
"""# Data cleaning
# check for missing values
print("\nVerificando valores ausentes:")
print(X.isnull().sum())"""

'# Data cleaning\n# check for missing values\nprint("\nVerificando valores ausentes:")\nprint(X.isnull().sum())'

## Pré-processamento dataset Student Performance

In [50]:
"""# creating classification targets
y_final = np.where(y['G3'] >= 10, 1, 0)  # Convertendo a nota final em uma classificação binária (aprovado/reprovado). Usando 10 como o limite de aprovação.
# 1 representa aprovado e 0 representa reprovado.
# Verificando a distribuição das classes
print("\nDistribuição das classes (Aprovado/Reprovado):")
print(pd.Series(y_final).value_counts()) # Contando as ocorrências de cada classe. usando pd.Series para criar uma série pandas a partir do array numpy"""




'# creating classification targets\ny_final = np.where(y[\'G3\'] >= 10, 1, 0)  # Convertendo a nota final em uma classificação binária (aprovado/reprovado). Usando 10 como o limite de aprovação.\n# 1 representa aprovado e 0 representa reprovado.\n# Verificando a distribuição das classes\nprint("\nDistribuição das classes (Aprovado/Reprovado):")\nprint(pd.Series(y_final).value_counts()) # Contando as ocorrências de cada classe. usando pd.Series para criar uma série pandas a partir do array numpy'

In [51]:
"""# The dataset seem unbalanced, so it has to be balanced

# tranforming text features into numerical features
X_final = pd.get_dummies(X, drop_first=True)  # Convertendo variáveis categóricas em variáveis dummy # drop_first=True evita a armadilha da variável fictícia
# Verificando as dimensões dos dados após a codificação
print("\nDimensões dos dados após a codificação:") 
print("Formato das features (X) antes do processamento:", X.shape)
print("Formato das features (X) após o processamento:", X_final.shape)
# Verificando as primeiras linhas dos dados codificados
print("\nPrimeiras linhas dos dados codificados:")
print(X_final.head())
print(X_final.dtypes)"""



'# The dataset seem unbalanced, so it has to be balanced\n\n# tranforming text features into numerical features\nX_final = pd.get_dummies(X, drop_first=True)  # Convertendo variáveis categóricas em variáveis dummy # drop_first=True evita a armadilha da variável fictícia\n# Verificando as dimensões dos dados após a codificação\nprint("\nDimensões dos dados após a codificação:") \nprint("Formato das features (X) antes do processamento:", X.shape)\nprint("Formato das features (X) após o processamento:", X_final.shape)\n# Verificando as primeiras linhas dos dados codificados\nprint("\nPrimeiras linhas dos dados codificados:")\nprint(X_final.head())\nprint(X_final.dtypes)'

## Spliting data into training and testing sets

In [52]:
"""X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42, stratify=y_final)  # Stratify -> mantém a proporção das classes no split

print(f"\nDimensões do conjunto de treino: {X_train.shape}")
print(f"Dimensões do conjunto de teste: {X_test.shape}")"""

'X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=42, stratify=y_final)  # Stratify -> mantém a proporção das classes no split\n\nprint(f"\nDimensões do conjunto de treino: {X_train.shape}")\nprint(f"Dimensões do conjunto de teste: {X_test.shape}")'

## Normalizing 

In [53]:
"""scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)  """

'scaler = StandardScaler()\nX_train_scaled = scaler.fit_transform(X_train)  \nX_test_scaled = scaler.transform(X_test)  '

## Using SMOTE to balance

In [54]:
"""# Using SMOTE to balance the dataset
from imblearn.over_sampling import SMOTE

print("Distribuição de classes no treino ANTES do SMOTE:")
print(pd.Series(y_train).value_counts())

# aplying SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)

# Verificando o balanceamento DEPOIS do SMOTE
print("\nDistribuição de classes no treino DEPOIS do SMOTE:")
print(pd.Series(y_train_resampled).value_counts())"""

'# Using SMOTE to balance the dataset\nfrom imblearn.over_sampling import SMOTE\n\nprint("Distribuição de classes no treino ANTES do SMOTE:")\nprint(pd.Series(y_train).value_counts())\n\n# aplying SMOTE\nsmote = SMOTE(random_state=42)\nX_train_resampled, y_train_resampled = smote.fit_resample(X_train_scaled, y_train)\n\n# Verificando o balanceamento DEPOIS do SMOTE\nprint("\nDistribuição de classes no treino DEPOIS do SMOTE:")\nprint(pd.Series(y_train_resampled).value_counts())'

## Training the dataset